# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
# your code here
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.models import load_model

In [ ]:
df=pd.read_csv(r'C:\DSML bootcamp\Week6\Day1\lab-neural-networks\your-code\tic-tac-toe.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
print(df.nunique())

In [ ]:
# Convert the categorical values to numeric using OneHotEncoder
X = df.drop('class', axis=1)
y = df['class']

encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X)

# Separate the inputs (X_encoded) and output (y)
# Convert target column to numeric as well
y = df['class'].astype(int)

# Normalize the input data
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(X_encoded)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
# your code here

# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# Build the model
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(),loss=SparseCategoricalCrossentropy(),metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest accuracy: {accuracy:.4f}")

# Save the model
model.save("tic-tac-toe.keras")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here

# Load the saved model
model = load_model("tic-tac-toe.keras")

# Pick a few random rows from the test set
num_samples = 5
indices = np.random.choice(len(X_test), size=num_samples, replace=False)
X_sample = X_test[indices]
y_true = y_test.iloc[indices]  # se y_test for um Series do pandas

# Make predictions
predictions = model.predict(X_sample)

# Convert softmax output to class labels (0 ou 1)
predicted_classes = np.argmax(predictions, axis=1)

# Show results
for i in range(num_samples):
    print(f"Sample {i+1}:")
    print(f"  True Label:      {y_true.iloc[i]}")
    print(f"  Predicted Label: {predicted_classes[i]}")
    print()

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

# 1. Build a deeper model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 2. Use Adam optimizer with custom learning rate
optimizer = Adam(learning_rate=0.0005)

# 3. Compile the model
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 4. Train the model with more epochs + validation
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\nTest accuracy: {accuracy:.4f}")

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# To improve the model's performance, several changes were implemented. First, 
# additional hidden layers were added to increase the model's capacity to learn more complex patterns. 
# Then, the learning rate of the optimizer was manually adjusted using the Adam optimizer with a lower rate (0.0005) to ensure more stable training. 
# The number of training epochs was also increased to give the model more time to learn. 
# Lastly, a validation split was introduced during training to monitor performance and avoid overfitting. 
# These adjustments aimed to reduce loss and boost accuracy.